In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install transformers

In [4]:
import torch

In [5]:
from transformers import pipeline

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [7]:
#Load pre_trained  GPT-2 MODEL andTokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
num_params=sum(p.numel() for p in model.parameters())
print(f"Number of parameters in the model: {num_params}")
#124.39 MIllion

Number of parameters in the model: 124439808


In [9]:
#Set the device to GPU if avaialble , else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [12]:
#Text generation parameters
prompt_text = "I can do"
max_length = 10
num_return_sequences = 1

In [14]:
#Generate text samples
generated_texts=[]
for _ in range(num_return_sequences):
    input_ids = tokenizer.encode(prompt_text, return_tensors="pt").to(device)
    output = model.generate(input_ids, max_length=max_length,
                            pad_token_id=tokenizer.eos_token_id, num_return_sequences=1,
                            temperature=0.7, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_texts.append(generated_text)
#print the generated text samples
for i , generated_text in enumerate(generated_texts):
    print(f"Generated Text {i+1}: {generated_text}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated Text 1: I can do this. I can't do that


In [29]:
#Replace this with your own text generation dataset
train_texts=["My name is sayiqa Bot.",
             "I can do anything.",
             "I am trained for giving a demo."]
#set the pad_token for the tokenizer
tokenizer.pad_token = tokenizer.eos_token
#tokenize the training texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt")
#Fine-tune  the gpt-2 model for text generation
optimizer=torch.optim.AdamW(model.parameters(), lr=1e-5)
model.train()
for epoch in range(300):  #replace 5 with the desired number of epochs
     outputs=model(**train_encodings, labels=train_encodings.input_ids)
     loss=outputs.loss
     loss.backward()
     optimizer.step()
     optimizer.zero_grad()
     #print(f"Epoch {epoch+1}/{300} Loss: {loss.item()}")
     print(f"Epoch {epoch+1} Loss: {loss.item()}")
 #save the fine-tuned model
output_dir="./fine_tuned_model_gpt2"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Epoch 1 Loss: 0.24689501523971558
Epoch 2 Loss: 0.06548915803432465
Epoch 3 Loss: 0.6429604887962341
Epoch 4 Loss: 0.11341797560453415
Epoch 5 Loss: 0.1528446227312088
Epoch 6 Loss: 0.05565555766224861
Epoch 7 Loss: 0.019710805267095566
Epoch 8 Loss: 0.3895359933376312
Epoch 9 Loss: 0.10550251603126526
Epoch 10 Loss: 0.1046992838382721
Epoch 11 Loss: 0.06513570994138718
Epoch 12 Loss: 0.4212592542171478
Epoch 13 Loss: 0.44065552949905396
Epoch 14 Loss: 0.42766672372817993
Epoch 15 Loss: 0.06598407030105591
Epoch 16 Loss: 0.33805808424949646
Epoch 17 Loss: 0.09824488312005997
Epoch 18 Loss: 0.29286062717437744
Epoch 19 Loss: 0.0563422329723835
Epoch 20 Loss: 0.9468763470649719
Epoch 21 Loss: 0.23102357983589172
Epoch 22 Loss: 0.428884357213974
Epoch 23 Loss: 0.6808977723121643
Epoch 24 Loss: 0.0448058657348156
Epoch 25 Loss: 0.07086797058582306
Epoch 26 Loss: 0.31346791982650757
Epoch 27 Loss: 0.0665762722492218
Epoch 28 Loss: 0.07388754189014435
Epoch 29 Loss: 0.08894233405590057
Epoch

('./fine_tuned_model_gpt2/tokenizer_config.json',
 './fine_tuned_model_gpt2/special_tokens_map.json',
 './fine_tuned_model_gpt2/vocab.json',
 './fine_tuned_model_gpt2/merges.txt',
 './fine_tuned_model_gpt2/added_tokens.json')

In [30]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
# Load the fine tuned GPT-2 model and tokenizer
output_dir="./fine_tuned_model_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model = GPT2LMHeadModel.from_pretrained(output_dir)
#set the device to GPU if available , else use CPU
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#Text generation parameters
prompt_text = "My name is"
max_length = 10
num_return_sequences = 1
#Generate text samples
generated_texts=[]
for _ in range(num_samples):
    input_ids = tokenizer.encode(prompt_text, return_tensors="pt").to(device)
    with torch.no_grad():
      output = model.generate(input_ids, max_length=max_length,
                            pad_token_id=tokenizer.eos_token_id, num_return_sequences=1,
                            temperature=0.7, no_repeat_ngram_size=2)
    #Control the randomness of the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_texts.append(generated_text)
#print the generated text samples
for i , text in enumerate(generated_texts):
    print(f"Generated Text {i+1}: {text}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated Text 1: My name is sayiqa Bot.
